> __Paper:__
Physics-informed neural networks: A deep learning framework for solving forward and
inverse problems involving nonlinear partial differential equations

> [Link](https://doi.org/10.1016/j.jcp.2018.10.045)

In [1]:
"""
@author: Maziar Raissi
"""

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.io
#from plotting import savefig, newfig
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

np.random.seed(1234)
tf.set_random_seed(1234)

import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:

class PhysicsInformedNN:
    # Initialize the class
    def __init__(self, x0, u0, x1, layers, dt, lb, ub, q):
        
        self.lb = lb
        self.ub = ub
        
        self.x0 = x0
        self.x1 = x1
        
        self.u0 = u0
        
        self.layers = layers
        self.dt = dt
        self.q = max(q,1)
    
        # Initialize NN
        self.weights, self.biases = self.initialize_NN(layers)
        
        # Load IRK weights
        tmp = np.float32(np.loadtxt('../../clones/PINNs/Utilities/IRK_weights/Butcher_IRK%d.txt' % (q), ndmin = 2))
        self.IRK_weights = np.reshape(tmp[0:q**2+q], (q+1,q))
        self.IRK_times = tmp[q**2+q:]
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        
        self.x0_tf = tf.placeholder(tf.float32, shape=(None, self.x0.shape[1]))
        self.x1_tf = tf.placeholder(tf.float32, shape=(None, self.x1.shape[1]))
        self.u0_tf = tf.placeholder(tf.float32, shape=(None, self.u0.shape[1]))
        self.dummy_x0_tf = tf.placeholder(tf.float32, shape=(None, self.q)) # dummy variable for fwd_gradients
        self.dummy_x1_tf = tf.placeholder(tf.float32, shape=(None, self.q+1)) # dummy variable for fwd_gradients
        
        self.U0_pred = self.net_U0(self.x0_tf) # N x (q+1)
        self.U1_pred, self.U1_x_pred= self.net_U1(self.x1_tf) # N1 x (q+1)
        
        self.loss = tf.reduce_sum(tf.square(self.u0_tf - self.U0_pred)) + \
                    tf.reduce_sum(tf.square(self.U1_pred[0,:] - self.U1_pred[1,:])) + \
                    tf.reduce_sum(tf.square(self.U1_x_pred[0,:] - self.U1_x_pred[1,:]))                     
        
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})
        
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        
        init = tf.global_variables_initializer()
        self.sess.run(init)
        
    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        
        #######################################################
        ##         Roozbeh
        #######################################################
        # The input is [None , 1], which is a column vector x
        # The middle layers are [None, 200]
        # The output is [None , q+1], which are calculated column 
        # vectors for Runge-Kutta steps plus u in the last column
        #######################################################
        num_layers = len(weights) + 1
        
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
    
    def fwd_gradients_0(self, U, x):        
        g = tf.gradients(U, x, grad_ys=self.dummy_x0_tf)[0]
        return tf.gradients(g, self.dummy_x0_tf)[0]
    
    def fwd_gradients_1(self, U, x):        
        g = tf.gradients(U, x, grad_ys=self.dummy_x1_tf)[0]
        return tf.gradients(g, self.dummy_x1_tf)[0]
    
    def net_U0(self, x):
        U1 = self.neural_net(x, self.weights, self.biases)
        #######################################################
        ##         Roozbeh
        #######################################################
        # select the last column of the output, which is 
        # the calculated U
        #######################################################
        U = U1[:,:-1]
        #######################################################
        ##         Roozbeh
        #######################################################
        # Finds U_x and U_xx by using the autograd. 
        # And finally the PDE and the Runge-Kuta steps using
        # Butcher tableau coeffs is calculated
        #######################################################
        U_x = self.fwd_gradients_0(U, x)
        U_xx = self.fwd_gradients_0(U_x, x)
        F = 5.0*U - 5.0*U**3 + 0.0001*U_xx
        U0 = U1 - self.dt*tf.matmul(F, self.IRK_weights.T)
        return U0

    def net_U1(self, x):
        U1 = self.neural_net(x, self.weights, self.biases)
        U1_x = self.fwd_gradients_1(U1, x)
        return U1, U1_x # N x (q+1)
    
    def callback(self, loss):
        print('Loss:', loss)
    
    def train(self, nIter):
        tf_dict = {self.x0_tf: self.x0, self.u0_tf: self.u0, self.x1_tf: self.x1,
                   self.dummy_x0_tf: np.ones((self.x0.shape[0], self.q)),
                   self.dummy_x1_tf: np.ones((self.x1.shape[0], self.q+1))}
        
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print
            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                print('It: %d, Loss: %.3e, Time: %.2f' % 
                      (it, loss_value, elapsed))
                start_time = time.time()
    
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss],
                                loss_callback = self.callback)
    
    def predict(self, x_star):
        
        U1_star = self.sess.run(self.U1_pred, {self.x1_tf: x_star})
                    
        return U1_star

In [33]:
x_star.shape

(512, 1)

In [30]:
model.IRK_weights.shape

(101, 100)

In [31]:
model.IRK_times.shape

(100, 1)

In [3]:
q = 100
layers = [1, 200, 200, 200, 200, q+1]
lb = np.array([-1.0])
ub = np.array([1.0])

N = 200

data = scipy.io.loadmat('../../clones/PINNs/main/Data/AC.mat')

t = data['tt'].flatten()[:,None] # T x 1
x = data['x'].flatten()[:,None] # N x 1
Exact = np.real(data['uu']).T # T x N

idx_t0 = 20
idx_t1 = 180
dt = t[idx_t1] - t[idx_t0]

# Initial data
noise_u0 = 0.0
idx_x = np.random.choice(Exact.shape[1], N, replace=False) 
x0 = x[idx_x,:]
u0 = Exact[idx_t0:idx_t0+1,idx_x].T
u0 = u0 + noise_u0*np.std(u0)*np.random.randn(u0.shape[0], u0.shape[1])


# Boudanry data
x1 = np.vstack((lb,ub))

# Test data
x_star = x

model = PhysicsInformedNN(x0, u0, x1, layers, dt, lb, ub, q)
model.train(10000)

U1_pred = model.predict(x_star)

error = np.linalg.norm(U1_pred[:,-1] - Exact[idx_t1,:], 2)/np.linalg.norm(Exact[idx_t1,:], 2)
print('Error: %e' % (error))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
It: 0, Loss: 4.052e+03, Time: 0.66
It: 10, Loss: 3.577e+03, Time: 0.12
It: 20, Loss: 3.567e+03, Time: 0.11
It: 30, Loss: 3.555e+03, Time: 0.11
It: 40, Loss: 3.544e+03, Time: 0.12
It: 50, Loss: 3.537e+03, Time: 0.11
It: 60, Loss: 3.521e+03, Time: 0.11
It: 70, Loss: 3.425e+03, Time: 0.11
It: 80, Loss: 2.578e+03, Time: 0.11
It: 90, Loss: 2.312e+03, Time: 0.11
It: 100, Loss: 1.744e+03, Time: 0.11
It: 110, Loss: 1.697e+03, Time: 0.11
It: 120, Loss: 1.

It: 1980, Loss: 5.377e+01, Time: 0.11
It: 1990, Loss: 4.898e+01, Time: 0.11
It: 2000, Loss: 4.853e+01, Time: 0.11
It: 2010, Loss: 4.759e+01, Time: 0.11
It: 2020, Loss: 4.650e+01, Time: 0.11
It: 2030, Loss: 4.607e+01, Time: 0.11
It: 2040, Loss: 4.857e+01, Time: 0.11
It: 2050, Loss: 6.820e+01, Time: 0.11
It: 2060, Loss: 4.578e+01, Time: 0.11
It: 2070, Loss: 4.740e+01, Time: 0.11
It: 2080, Loss: 4.588e+01, Time: 0.11
It: 2090, Loss: 1.180e+02, Time: 0.11
It: 2100, Loss: 5.086e+01, Time: 0.11
It: 2110, Loss: 4.481e+01, Time: 0.11
It: 2120, Loss: 4.383e+01, Time: 0.11
It: 2130, Loss: 4.335e+01, Time: 0.11
It: 2140, Loss: 4.301e+01, Time: 0.11
It: 2150, Loss: 4.279e+01, Time: 0.11
It: 2160, Loss: 4.251e+01, Time: 0.11
It: 2170, Loss: 4.223e+01, Time: 0.11
It: 2180, Loss: 4.199e+01, Time: 0.11
It: 2190, Loss: 4.181e+01, Time: 0.11
It: 2200, Loss: 4.675e+01, Time: 0.11
It: 2210, Loss: 4.860e+01, Time: 0.11
It: 2220, Loss: 4.866e+01, Time: 0.11
It: 2230, Loss: 4.172e+01, Time: 0.11
It: 2240, Lo

It: 4140, Loss: 2.499e+01, Time: 0.11
It: 4150, Loss: 2.286e+01, Time: 0.11
It: 4160, Loss: 2.039e+01, Time: 0.11
It: 4170, Loss: 2.118e+01, Time: 0.11
It: 4180, Loss: 2.456e+01, Time: 0.11
It: 4190, Loss: 2.460e+01, Time: 0.11
It: 4200, Loss: 2.227e+01, Time: 0.11
It: 4210, Loss: 1.973e+01, Time: 0.11
It: 4220, Loss: 1.968e+01, Time: 0.11
It: 4230, Loss: 1.929e+01, Time: 0.11
It: 4240, Loss: 1.960e+01, Time: 0.11
It: 4250, Loss: 2.794e+01, Time: 0.11
It: 4260, Loss: 1.958e+01, Time: 0.11
It: 4270, Loss: 1.956e+01, Time: 0.11
It: 4280, Loss: 2.006e+01, Time: 0.11
It: 4290, Loss: 1.983e+01, Time: 0.11
It: 4300, Loss: 8.411e+01, Time: 0.11
It: 4310, Loss: 3.104e+01, Time: 0.11
It: 4320, Loss: 2.114e+01, Time: 0.11
It: 4330, Loss: 2.059e+01, Time: 0.11
It: 4340, Loss: 1.944e+01, Time: 0.11
It: 4350, Loss: 1.927e+01, Time: 0.11
It: 4360, Loss: 4.916e+01, Time: 0.11
It: 4370, Loss: 2.983e+01, Time: 0.11
It: 4380, Loss: 2.253e+01, Time: 0.11
It: 4390, Loss: 1.965e+01, Time: 0.11
It: 4400, Lo

It: 6300, Loss: 9.884e+00, Time: 0.11
It: 6310, Loss: 9.867e+00, Time: 0.11
It: 6320, Loss: 9.487e+00, Time: 0.11
It: 6330, Loss: 1.409e+01, Time: 0.11
It: 6340, Loss: 1.906e+01, Time: 0.11
It: 6350, Loss: 1.505e+01, Time: 0.11
It: 6360, Loss: 1.220e+01, Time: 0.11
It: 6370, Loss: 9.532e+00, Time: 0.11
It: 6380, Loss: 9.565e+00, Time: 0.11
It: 6390, Loss: 1.092e+01, Time: 0.11
It: 6400, Loss: 1.477e+01, Time: 0.11
It: 6410, Loss: 1.063e+01, Time: 0.11
It: 6420, Loss: 1.504e+01, Time: 0.11
It: 6430, Loss: 1.902e+01, Time: 0.11
It: 6440, Loss: 1.180e+01, Time: 0.11
It: 6450, Loss: 9.877e+00, Time: 0.11
It: 6460, Loss: 9.136e+00, Time: 0.11
It: 6470, Loss: 9.445e+00, Time: 0.11
It: 6480, Loss: 9.976e+00, Time: 0.11
It: 6490, Loss: 2.215e+01, Time: 0.11
It: 6500, Loss: 2.983e+01, Time: 0.11
It: 6510, Loss: 1.305e+01, Time: 0.11
It: 6520, Loss: 1.098e+01, Time: 0.11
It: 6530, Loss: 1.008e+01, Time: 0.11
It: 6540, Loss: 9.130e+00, Time: 0.11
It: 6550, Loss: 8.926e+00, Time: 0.11
It: 6560, Lo

It: 8460, Loss: 6.676e+00, Time: 0.11
It: 8470, Loss: 5.853e+00, Time: 0.11
It: 8480, Loss: 5.660e+00, Time: 0.11
It: 8490, Loss: 5.948e+00, Time: 0.11
It: 8500, Loss: 2.445e+01, Time: 0.11
It: 8510, Loss: 1.496e+01, Time: 0.11
It: 8520, Loss: 7.481e+00, Time: 0.11
It: 8530, Loss: 6.132e+00, Time: 0.11
It: 8540, Loss: 6.837e+00, Time: 0.11
It: 8550, Loss: 9.813e+00, Time: 0.11
It: 8560, Loss: 6.173e+00, Time: 0.11
It: 8570, Loss: 7.142e+00, Time: 0.11
It: 8580, Loss: 6.448e+00, Time: 0.11
It: 8590, Loss: 9.332e+00, Time: 0.11
It: 8600, Loss: 6.585e+00, Time: 0.11
It: 8610, Loss: 1.224e+01, Time: 0.11
It: 8620, Loss: 7.151e+00, Time: 0.11
It: 8630, Loss: 6.611e+00, Time: 0.11
It: 8640, Loss: 7.633e+00, Time: 0.11
It: 8650, Loss: 7.023e+00, Time: 0.11
It: 8660, Loss: 5.360e+00, Time: 0.11
It: 8670, Loss: 6.857e+00, Time: 0.11
It: 8680, Loss: 2.146e+01, Time: 0.11
It: 8690, Loss: 1.341e+01, Time: 0.11
It: 8700, Loss: 1.243e+01, Time: 0.11
It: 8710, Loss: 6.591e+00, Time: 0.11
It: 8720, Lo

...

Loss: 0.007847138
Loss: 0.007844164
Loss: 0.007840508
Loss: 0.0078384075
Loss: 0.007834125
Loss: 0.0078320205
Loss: 0.0078297965
Loss: 0.007828125
Loss: 0.007826278
Loss: 0.0078243045
Loss: 0.007822918
Loss: 0.00781918
Loss: 0.007835621
Loss: 0.007818412
Loss: 0.007815163
Loss: 0.007812646
Loss: 0.007808972
Loss: 0.007805734
Loss: 0.0078070643
Loss: 0.00780381
Loss: 0.0078004007
Loss: 0.0077986824
Loss: 0.0077955886
Loss: 0.0077954535
Loss: 0.0077946912
Loss: 0.0077932603
Loss: 0.0077894335
Loss: 0.0077859596
Loss: 0.0077841515
Loss: 0.0077811973
Loss: 0.0077795684
Loss: 0.0077763554
Loss: 0.0077727246
Loss: 0.0077749197
Loss: 0.0077710655
Loss: 0.0077686347
Loss: 0.00776734
Loss: 0.0077650123
Loss: 0.007761436
Loss: 0.007781342
Loss: 0.0077604945
Loss: 0.007757789
Loss: 0.0077561582
Loss: 0.007754008
Loss: 0.007753583
Loss: 0.0077504856
Loss: 0.0077485126
Loss: 0.007747103
Loss: 0.00774558
Loss: 0.0077414843
Loss: 0.0077631855
Loss: 0.0077410885
Loss: 0.0077396836
Loss: 0.007737267
Lo

Loss: 0.0069995373
Loss: 0.0069984915
Loss: 0.0069978274
Loss: 0.0069952547
Loss: 0.006994322
Loss: 0.006993188
Loss: 0.00699146
Loss: 0.00698901
Loss: 0.0069872467
Loss: 0.0069842367
Loss: 0.0069833663
Loss: 0.006981013
Loss: 0.0069784634
Loss: 0.006988729
Loss: 0.0069773216
Loss: 0.0069747767
Loss: 0.0069732554
Loss: 0.0069713295
Loss: 0.0069696866
Loss: 0.006968047
Loss: 0.006966718
Loss: 0.0069655697
Loss: 0.0069637415
Loss: 0.006963274
Loss: 0.006960516
Loss: 0.0069590462
Loss: 0.006957859
Loss: 0.0069560246
Loss: 0.0069649774
Loss: 0.0069549936
Loss: 0.0069528962
Loss: 0.0069513153
Loss: 0.006949095
Loss: 0.006947609
Loss: 0.0069455854
Loss: 0.0069440957
Loss: 0.0069422475
Loss: 0.0069396473
Loss: 0.0069553778
Loss: 0.0069389394
Loss: 0.0069368836
Loss: 0.0069360915
Loss: 0.0069333585
Loss: 0.006929588
Loss: 0.0069246935
Loss: 0.006973787
Loss: 0.0069239205
Loss: 0.0069212797
Loss: 0.006919197
Loss: 0.0069197766
Loss: 0.006917931
Loss: 0.006917244
Loss: 0.0069152387
Loss: 0.00691

Loss: 0.00623516
Loss: 0.006233447
Loss: 0.006231621
Loss: 0.0062309597
Loss: 0.006228335
Loss: 0.0062278556
Loss: 0.0062264004
Loss: 0.006225834
Loss: 0.006224054
Loss: 0.006222962
Loss: 0.0062222597
Loss: 0.006220488
Loss: 0.006219407
Loss: 0.0062178713
Loss: 0.006216695
Loss: 0.0062155016
Loss: 0.0062134853
Loss: 0.00621358
Loss: 0.00621196
Loss: 0.0062107295
Loss: 0.0062096384
Loss: 0.006208327
Loss: 0.006206416
Loss: 0.006203902
Loss: 0.0062029213
Loss: 0.006201118
Loss: 0.006200103
Loss: 0.0061992933
Loss: 0.0061986293
Loss: 0.0061974595
Loss: 0.0061954106
Loss: 0.0061930018
Loss: 0.006192557
Loss: 0.0061911386
Loss: 0.006189381
Loss: 0.0061886343
Loss: 0.0061870534
Loss: 0.006186192
Loss: 0.006183836
Loss: 0.006182763
Loss: 0.0061817905
Loss: 0.00618088
Loss: 0.006179816
Loss: 0.006177617
Loss: 0.0061810696
Loss: 0.006176924
Loss: 0.00617589
Loss: 0.0061751218
Loss: 0.006173375
Loss: 0.00617025
Loss: 0.0061713373
Loss: 0.0061696414
Loss: 0.0061664945
Loss: 0.006164446
Loss: 0.00

Loss: 0.005647083
Loss: 0.0056455047
Loss: 0.005643935
Loss: 0.00564325
Loss: 0.0056425245
Loss: 0.0056415326
Loss: 0.0056396346
Loss: 0.0056380713
Loss: 0.0056363936
Loss: 0.005634156
Loss: 0.005632477
Loss: 0.0056310897
Loss: 0.0056298636
Loss: 0.0056280745
Loss: 0.0056269467
Loss: 0.0056254775
Loss: 0.005625962
Loss: 0.005624779
Loss: 0.005623613
Loss: 0.0056227897
Loss: 0.005621528
Loss: 0.005620624
Loss: 0.0056194593
Loss: 0.0056187534
Loss: 0.0056175147
Loss: 0.005616291
Loss: 0.005615138
Loss: 0.0056139743
Loss: 0.0056127138
Loss: 0.005612327
Loss: 0.005610737
Loss: 0.0056094215
Loss: 0.0056091086
Loss: 0.0056078257
Loss: 0.0056059067
Loss: 0.0056047677
Loss: 0.005603771
Loss: 0.0056026466
Loss: 0.00560123
Loss: 0.0055995113
Loss: 0.005598096
Loss: 0.00559758
Loss: 0.005595951
Loss: 0.005594803
Loss: 0.005593581
Loss: 0.005591324
Loss: 0.0055891527
Loss: 0.005588617
Loss: 0.005586233
Loss: 0.0055849464
Loss: 0.0055841105
Loss: 0.0055830088
Loss: 0.0055822707
Loss: 0.005579071
Lo

Error: 4.884286e-03


In [4]:
saver = tf.train.Saver()
saver.save(model.sess, 'Allen_Cahn_model', global_step=0)

'Allen_Cahn_model-0'

In [5]:
model.U1_pred.name

'Add_10:0'

In [6]:
model.x1_tf.name

'Placeholder_1:0'